## Walmart leetcode

### 697. Degree of an Array

Given a non-empty array of non-negative integers nums, the degree of this array is defined as the maximum frequency of any one of its elements.

Your task is to find the smallest possible length of a (contiguous) subarray of nums, that has the same degree as nums.


In [7]:
nums = [1,2,2,3,1] # answer = 2

In [8]:
# Raw solution
class Solution(object):
    def findShortestSubArray(nums):
        """
        :type nums: List[int]
        :rtype: int
        """
        dict_index = {}
        dict_count = {}
        max_count = 1
        max_length = 1

        for i in range(len(nums)):
            
            if nums[i] not in dict_count:
                dict_count[nums[i]] = 1
                dict_index[nums[i]] = [i,i]
                
            elif nums[i] in dict_count:
                dict_count[nums[i]] += 1
                dict_index[nums[i]][1] = i
            
            if dict_count[nums[i]] > max_count:
                max_count = dict_count[nums[i]]
                max_length = dict_index[nums[i]][-1] - dict_index[nums[i]][0] + 1
                
            elif dict_count[nums[i]] == max_count:
                max_length = min(max_length, dict_index[nums[i]][-1] - dict_index[nums[i]][0]+1)
                
        
        return max_length
                

In [10]:
Solution.findShortestSubArray(nums)

2

In [12]:
class Solution:
    def findShortestSubArray(nums):
        left,right,count = {},{},{} #设置一个记住左边和右边的dict, 还有count和dict.思想和raw solution 里面的一样
        for i,x in enumerate(nums):
            if x not in left: 
                left[x] = i 
            right[x] = i
            count[x] = count.get(x,0)+1 #用get(key，default)而不是dict[key]的好处是，如果key不在里面，会自动返回default值
        
        ans = len(nums)
        degree = max(count.values())
        for x in count: # 这里用了第二个loop来比较是在最大freq的情况下的最短subarray
            if count[x]==degree:
                ans = min(ans, right[x]-left[x]+1)
        
        return(ans)

In [13]:
Solution.findShortestSubArray(nums)

2

### 355. Design Twitter

Design a simplified version of Twitter where users can post tweets, follow/unfollow another user, and is able to see the 10 most recent tweets in the user's news feed.

Implement the Twitter class:

Twitter() Initializes your twitter object.
void postTweet(int userId, int tweetId) Composes a new tweet with ID tweetId by the user userId. Each call to this function will be made with a unique tweetId.
List<Integer> getNewsFeed(int userId) Retrieves the 10 most recent tweet IDs in the user's news feed. Each item in the news feed must be posted by users who the user followed or by the user themself. Tweets must be ordered from most recent to least recent.
void follow(int followerId, int followeeId) The user with ID followerId started following the user with ID followeeId.
void unfollow(int followerId, int followeeId) The user with ID followerId started unfollowing the user with ID followeeId.
    

In [6]:
# Raw solution (beat 98%, 这道题主要复习class的构建，和queue还有stack的用法)
class Twitter:

    def __init__(self):
        
        self.user_userId = []
        self.user_postId = []
        self.user_follow = {}
        

    def postTweet(self, userId: int, tweetId: int) -> None:
        self.user_postId.insert(0, tweetId)  #创建tweet顺序，后发的放在前面
        self.user_userId.insert(0, userId)   #同上
        

    def getNewsFeed(self, userId: int):

        all_user = [userId] + self.user_follow.get(userId,[]) #找到全部user,包括自己和followee
        newsFeed_index = [index for index, x in enumerate(self.user_userId) if x in all_user] #找user 对应index

        newsFeed = [news for index, news in enumerate(self.user_postId) if index in newsFeed_index] #找到index对应的newfeed
        if len(newsFeed) > 10:
            newsFeed = newsFeed[:10] #筛出最新的10条
        
        return newsFeed
        

    def follow(self, followerId: int, followeeId: int) -> None:
        if followerId not in self.user_follow:
            self.user_follow[followerId] = [followeeId]
        else:
            self.user_follow[followerId].append(followeeId) #把 followee放在dict相关位置的list里面
        

    def unfollow(self, followerId: int, followeeId: int) -> None:
        if followeeId in self.user_follow.get(followerId,[]):
            self.user_follow[followerId].pop(self.user_follow[followerId].index(followeeId)) #把 unfollowee pop出去
        

### 790. Domino and Tromino Tiling

You have two types of tiles: a 2 x 1 domino shape and a tromino shape. You may rotate these shapes.


Given an integer n, return the number of ways to tile an 2 x n board. Since the answer may be very large, return it modulo 109 + 7.

In a tiling, every square must be covered by a tile. Two tilings are different if and only if there are two 4-directionally adjacent cells on the board such that exactly one of the tilings has both squares occupied by a tile.

- Fully covered board: All tiles on board are covered by a domino or a tromino.
- Partially covered board: Same as a fully covered board, except leave the tile in the upper-right corner (the top row of the rightmost column) uncovered. 
- Note, a board with only the lower-right corner uncovered is also considered "partially covered." However, as we will discover soon, we do not need to keep track of which corner is uncovered because of symmetry.
- f(k): The number of ways to fully cover a board of width kk.
- p(k): The number of ways to partially cover a board of width kk.

In [14]:
# 这道题只能用dynamic programming去解。其实就是找规律。
# 长度为 n 的board的组合方法是长度为 n-1 的组合方法 （加一条竖的domino）加上 n -2 组合方法 （加两条横的domino） 
# 再加上 2乘以 n-1 的partial board组合方法 （partial board加一个L）.乘以2是因为可以上面或者下面缺一个的partial board

# 长度为 n 的partial board的组合方法其实就是长度为 n-2的full board (上或下对称的两种加L的方法) 
# 再加上 partial board n-1的组合方法 （加一条横的domino）

In [15]:
# 用list的记录
class Solution:
    def numTilings(self, n: int) -> int:
        MOD = 1_000_000_007
        f = n * [0]
        p = n * [0]
        
        if n == 1:
            return 1
        elif n == 2:
            return 2
        
        f[0] = 1
        f[1] = 2
        p[1] = 1
        
        for i in range(2, n):

            f[i] = (f[i-1] + f[i-2] + 2*p[i-1]) % MOD
            p[i] = (p[i-1] + f[i-2]) % MOD
            
 
        return f[n]

### 49. Group Anagrams
Given an array of strings strs, group the anagrams together. You can return the answer in any order.

An Anagram is a word or phrase formed by rearranging the letters of a different word or phrase, typically using all the original letters exactly once.

In [8]:
# 实际上就是找到每个letter count的dict.然后根据这个dict来判断两个string是不是anagram

class Solution(object):
    def groupAnagrams(self, strs):
        """
        :type strs: List[str]
        :rtype: List[List[str]]
        """
        
        distinct_set = []
        results = []
        
        def string_letter_count(string):
            res = {}
            for l in string:
                if l not in res:
                    res[l] = 1
                else:
                    res[l] += 1
            return res
                    
        
        for string in strs:
            distinct_letters = string_letter_count(string)
            
            if distinct_letters not in distinct_set:
                distinct_set.append(distinct_letters)
                results.append([string])
            elif distinct_letters in distinct_set:
                idx = distinct_set.index(distinct_letters)
                results[idx].append(string)
            

        return results
        

##### collections.defaultdict()介绍:

Usually, a Python dictionary throws a KeyError if you try to get an item with a key that is not currently in the dictionary. The defaultdict in contrast will simply create any items that you try to access (provided of course they do not exist yet). To create such a "default" item, it calls the function object that you pass to the constructor (more precisely, it's an arbitrary "callable" object, which includes function and type objects). For the first example, default items are created using int(), which will return the integer object 0. For the second example, default items are created using list(), which returns a new empty list object.

In [13]:
# 找26个字母对应count的列表。

# collections.defaultdict使用可以让效率更高
import collections
class Solution:
    def groupAnagrams(self, strs):
        ans = collections.defaultdict(list)
        for s in strs:
            count = [0] * 26
            for c in s:
                count[ord(c) - ord('a')] += 1 #关键是trick是用ord()找到字母在字母表里的顺序位置
            ans[tuple(count)].append(s)
        
        return ans.values()

### 22. Generate Parentheses

Given n pairs of parentheses, write a function to generate all combinations of well-formed parentheses.

In [17]:
# 典型的排列组合题。通过recursive function 生成所有的排列，然后设置条件return

class Solution:
    def generateParenthesis(self, n: int):
        res = []
        
        def helper(n_left, n_right, par):
            if n_left == 0 and n_right == 0: # 条件1：所有左右括号都完美用完
                res.append(par) 
            elif n_left < 0 or n_right < 0: #条件2：括号过度使用
                return
            elif n_left > n_right: #条件3：剩下的左括号多于右括号（这时使用的右括号多于左括号，所以括号排列不成立）
                return
            
            par += '('
            helper(n_left-1, n_right, par)
            par = par[:-1]
                    
            par += ')' 
            helper(n_left, n_right-1, par)
            par = par[:-1]
            
             
            
            
        helper(n,n,'')
        
        return(res)

#### 练习：定义一个叫做 is_palindrome的方程

In [67]:
# little pratice determine if a string is a parlindromic
s = 'abcba'
d = 'sddfe'

def check_parlin(s):
    n = len(s)
    if n % 2 == 0:
        left = s[:n//2]
        right = s[n//2:]
    elif n % 2 == 1:
        left = s[:n//2+1]
        right = s[n//2+2:]
    
    n_right = len(right)
    right_rev = [right[n_right - i -1] for i in range(n_right)]
    
    print(left, right_rev)
    
    return(left == right_rev)


In [68]:
## 方法1
def check_parlin(s):
    list_s = list(s)
    n = len(list_s)
    list_rev_s = [list_s[n-1-i] for i in range(n)]
    
    return(list_s == list_rev_s)

## 方法2 (two pointer的方法)
def is_parlin(s): 
    list_s = list(s)
    n = len(list_s)
    left = 0
    right = n - 1
    
    while left < right:
        print('left',left)
        print('right',right)
    
        if list_s[left] != list_s[right]:
            return False
        
        left = left + 1
        right = right - 1
    return True

### 22. Generate Parentheses

Given a string s, return the longest palindromic substring in s.

解法1: brute force. 定义一个function来判断是不是palindromic, 用两个指针loop所有的substring来进行判断是不是最长. 时间复杂度O（n^3)
解法2：DP， 中心思想是以一个点（奇数palindromic），或者两个点（偶数palindromic)来往两边扩展。一直括到最大的palindromic为止。 这样时间复杂度是O(n^2)

In [69]:
# brute force方法
class Solution:
    def longestPalindrome(self, s: str) -> str:
        
        size = len(s)
        
        def check_parlin(s):
            list_s = list(s)
            n = len(list_s)
            list_rev_s = [list_s[n-1-i] for i in range(n)]

            return(list_s == list_rev_s)
        
        max_parlin = 0
        for i in range(size):
            for j in range(size):
                substr = s[i:j+1]

                if check_parlin(substr) and len(substr) > max_parlin:
                    max_parlin = max(max_parlin, len(substr))
                    res = substr

        return(res)


In [70]:
# DP方法
class Solution:
    def longestPalindrome(self, s: str) -> str:
        
        def getlen(s, i, j):
            while i >= 0 and j < len(s) and s[i] == s[j]:
                i -= 1
                j += 1
            
            return(j - i - 1)
        
        max_length = 0
        for i in range(len(s)):
            
            length = max(getlen(s, i,i), getlen(s, i,i+1))
            
            if length > max_length:
                start = i - (length-1) //2
                end = start + length

                    
                res = s[start:end]
                max_length = max(length, max_length)
                
        print(max_length)
        return(res)
        

#### 练习：reverse一个list 或者string

In [6]:
def reverse_list(l):
    
    n = len(l)
    left = 0
    right = n - 1
    while left < right:
        temp = l[left]
        l[left] = l[right]
        l[right] = temp
        left += 1
        right -= 1
        
    return(l)

def reverse_string(s):
    n = len(s)
    s = list(s)
    l = 0
    r = n - 1
    while l < r:
        temp = s[l]
        s[l] = s[r]
        s[r] = temp
        l += 1
        r -= 1
    
    return(''.join(s))

In [7]:
example = [1,2,'a','b','6',4]

print(reverse_list(example))

[4, '6', 'b', 'a', 2, 1]


In [8]:
example_string = 'adfw323dsda'

print(reverse_string(example_string))

adsd323wfda


### 322. Coin Change

You are given an integer array coins representing coins of different denominations and an integer amount representing a total amount of money.

Return the fewest number of coins that you need to make up that amount. If that amount of money cannot be made up by any combination of the coins, return -1.

You may assume that you have an infinite number of each kind of coin.

In [9]:
class Solution(object):
    def coinChange(self, coins, amount):
        """
        :type coins: List[int]
        :type amount: int
        :rtype: int
        """
        self.res = []
        
        coins.sort(reverse = True)
      
        def search(i, coins, rest_amount, count):
            
            if self.res and count >= min(self.res): # pruning 剪枝
                return
            
            if rest_amount == 0:
                self.res.append(count)
                return
            
            if rest_amount <0:
                return
            
            for i in range(i, len(coins)):
                search(i, coins, rest_amount - coins[i], count + 1)
                
            return
                
            
        search(0, coins, amount, 0)

        output = min(self.res) if self.res else -1
            
        return output      
                 

### 253. Meeting Rooms II

Given an array of meeting time intervals intervals where intervals[i] = [starti, endi], return the minimum number of conference rooms required.

In [11]:
# 解题逻辑
# sort start and sort end
# start
# | |  |  |
# i
# end
#     |  |   |  |
#     end
# if i < end: meeting_room += 1
# if i > end: end +=1

# 核心思想是用一个buffer（就是i和end间的距离）来记录最大房间。如果buffer被用完说明当前overlap的数量要大于buffer，所有res 加1.反之则不用

class Solution(object):
    def minMeetingRooms(self, intervals):
        """
        :type intervals: List[List[int]]
        :rtype: int
        """
        
        
        n = len(intervals)
        
        start = n*[0]
        end = n*[0]
        
        for i in range(n):
            start[i] = intervals[i][0]
            end[i] = intervals[i][1]
            
        start.sort()
        end.sort()

        res = 0
        end_i = 0
        for i in range(n):
            if start[i] < end[end_i]:
                res += 1
            else:
                end_i += 1
                
        
        return res

### 122. Best Time to Buy and Sell Stock II
You are given an integer array prices where prices[i] is the price of a given stock on the ith day.

On each day, you may decide to buy and/or sell the stock. You can only hold at most one share of the stock at any time. However, you can buy it then immediately sell it on the same day.

Find and return the maximum profit you can achieve.

In [ ]:
# 买股票的题目的关键是取出那些增长的片段。就是利益最大化


class Solution(object):
    def maxProfit(self, prices):
        """
        :type prices: List[int]
        :rtype: int
        """
        
        # buy, unbuy, sell 
        profit = 0
        for i in range(1, len(prices)):
            delta = prices[i] - prices[i-1]
            if delta > 0:
                profit += delta
                
        
        return profit

### 560. Subarray Sum Equals K

Given an array of integers nums and an integer k, return the total number of continuous subarrays whose sum equals to k.

In [ ]:
# 这是一个超时解达，利用了Prefix sum的思想。prefix_sum_j - prefix_sum_i + num_i = sum(num[i,j])
class Solution(object):
    def subarraySum(self, nums, k):
        """
        :type nums: List[int]
        :type k: int
        :rtype: int
        """
        presum = len(nums)*[0]
        presum[0] = nums[0]
        for i in range(1, len(nums)):
            presum[i] = nums[i] + presum[i-1]
            
        count = 0

        for i in range(len(nums)):
            for j in range(i, len(nums)):
                if presum[j] - presum[i] + nums[i] == k:
                    count += 1
                    
        return count

In [12]:
matrix = [["1","0","1","0","0"],["1","0","1","1","1"],["1","1","1","1","1"],["1","0","0","1","0"]]

In [15]:
for i in range(5,6):
    print(i)

5


In [64]:
check_parlin(d)

False

In [65]:
s

'abcba'

In [66]:
is_parlin(s)

left 0
right 4
left 1
right 3


True

In [61]:
s[1:2]

'b'

In [21]:
s[:2]

'ab'